**Muhammad Abu Bakar**

In [ ]:
!pip install -qU langchain
!pip install -qU openai
!pip install -qU langchain-community
!pip install -qU langchain-experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 66.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.0/209.0 kB 13.5 MB/s eta 0:00:00


In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.chains import LLMChain, LLMMathChain, TransformChain, SequentialChain
from langchain.chains.router import MultiPromptChain
from langchain.callbacks import get_openai_callback

In [ ]:
from getpass import getpass

In [ ]:
OPENAI_API_KEY = getpass()

In [ ]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
from langchain.chains import SequentialChain

In [ ]:
!pip install -qU langchain-experimental

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.6/151.6 kB 5.0 MB/s eta 0:00:00


In [ ]:
from langchain_experimental.llm_bash.base import LLMBashChain
from langchain.prompts.prompt import PromptTemplate
from langchain_experimental.llm_bash.prompt import BashOutputParser
from langchain_experimental.llm_bash.base import LLMBashChain

In [ ]:
!pip install -qU langchain openai tiktoken

In [ ]:
import tiktoken

In [ ]:
llm = OpenAI(openai_api_key = OPENAI_API_KEY)

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory

## **Summary Buffer Memory**

In [ ]:
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)

## **Tranform Chain Functions**

In [ ]:
def transform_func_a(inputs: dict)-> dict:
  question = inputs["input"]
  answer = inputs["text"]
  substring_list = answer.split(':')
  memory.save_context({"input": question}, {"output": substring_list[1]})
  return {"history_a": inputs}

In [ ]:
def transform_func_b(inputs: dict)-> dict:
  question = inputs["input"]
  answer = inputs["text"]
  memory.save_context({"input": question}, {"output": answer})
  return {"history_b": inputs}

In [ ]:
def transform_func_c(inputs: dict)-> dict:
  question = inputs["input"]
  feed_back = memory.load_memory_variables(inputs = [])['history']
  question = f"{feed_back} \n{question}"
  return {"history_c": question}

## **Prompt Info**

In [ ]:
prompt_infos = [
    {
        "name": "bash",
        "description": "Good for writing bash scripts",
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
    },
]

## **Transform Chains**

In [ ]:
transform_chain_one = TransformChain(input_variables=["text"], output_variables=["history_a"], transform=transform_func_a)

In [ ]:
transform_chain_two = TransformChain(input_variables=["text"], output_variables=["history_b"], transform=transform_func_b)

In [ ]:
transform_chain_three = TransformChain(input_variables=["input"], output_variables=["history_c"], transform=transform_func_c)

## **LLM MATH CHAIN**

In [ ]:
llm_math = LLMMathChain(llm=llm, input_key = "input", output_key = "text", verbose=False)

## **LLM BASH CHAIN**

In [ ]:
llm_bash = LLMBashChain(llm=llm, input_key = "input", output_key ="text",verbose=True)

## **Destination Chains**

In [ ]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    if name == "math":
      chain = SequentialChain(chains=[transform_chain_three, llm_math, transform_chain_one], input_variables=["input"], output_variables=["text"])
      destination_chains[name] = chain
    if name == "bash":
      chain = SequentialChain(chains=[transform_chain_three, llm_bash, transform_chain_two], input_variables=["input"], output_variables=["text"])
      destination_chains[name] = chain
default_chain = ConversationChain(llm=llm, memory = memory, output_key = "text", verbose=False)

## **LLMRouterChain**

In [ ]:
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

## **MultiPromptChain**

In [ ]:
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=False,
)

## **Function to Activate Chat Bot**

In [ ]:
def chat_call_function():
  input_from_user = input(f"To exit write exit:\nHuman: ")
  if  input_from_user.lower() == "exit":
    print("Exiting recursion.")
    return
  result = chain.run(input_from_user)
  print(f"\nAI: {result}\n")
  chat_call_function()

## **Complete Chat**

In [ ]:
chat_call_function()

To exit write exit:
Human: Write me bash script of "for loop".


> Entering new LLMBashChain chain...
Write me a bash script of a 'for loop' to print out the numbers 1 to 10

```bash
for i in {1..10}; do
  echo $i
done
```
Code: ['for i in {1..10}; do', '  echo $i', 'done']
Answer: Command 'for i in {1..10}; do;  echo $i;done' returned non-zero exit status 2.
> Finished chain.

AI: Command 'for i in {1..10}; do;  echo $i;done' returned non-zero exit status 2.

To exit write exit:
Human: The square of 177 is

AI: Answer: 31329

To exit write exit:
Human: The cube root of 67

AI: Answer: 4.0615481004456795

To exit write exit:
Human: nice, give me a summary of our chat.

AI:  We discussed Islamabad and Karachi as two cities in Pakistan, and you asked me to write a bash script of a for loop, which returned a non-zero exit status 2. You then asked me to calculate the square root of 138, to which I responded with 11.74734012447073; the square of 156, to which I responded with 24336; and the

## **Summary of History**

In [ ]:
feed_back = memory.load_memory_variables(inputs = [])['history']
print(feed_back)

System: 
The human asked the AI to write a bash script for a for loop, which returned a non-zero exit status 2. The AI was then asked to calculate the square root of 138, and responded with 11.74734012447073. The human then asked the AI to write a bash script for a while loop, which again returned a non-zero exit status 2. The human then requested that the AI calculate the square of 156, to which the AI responded with 24336, and when asked to name any two cities in Pakistan, the AI responded with Islamabad and Karachi. The AI confirmed that they had discussed Islamabad and Karachi earlier, and also confirmed that the human had asked for a for loop and square root calculation. The AI then responded with the square of 177 and cube root of 67, both correctly.
Human: Thanks
AI:  You're welcome! Is there anything else I can help you with?


## **No of Used Token**

In [ ]:
tokenizer = tiktoken.encoding_for_model('text-davinci-003')
print(f'Summary Buffer memory conversation length: {len(tokenizer.encode(feed_back))}')

Summary Buffer memory conversation length: 191
